In [1]:
%%script echo skipping....
%conda install jupyter pytorch
%pip install 'git+https://github.com/deepset-ai/haystack.git#main' sentence-transformers 'txtai[pipeline-data]' qdrant-haystack gradio

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/deepset-ai/haystack.git to /private/var/folders/43/13h_2sgd3bxflq4m898ym8bm0000gn/T/pip-req-build-kqmxrynq
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /private/var/folders/43/13h_2sgd3bxflq4m898ym8bm0000gn/T/pip-req-build-kqmxrynq
  Resolved https://github.com/deepset-ai/haystack.git to commit e6d6ce1c73d62e087cf197e82e831e4216fc33c9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
%%script echo skipping.....
# !docker run --rm -p 6333:6333 -p 6334:6334 -v $(pwd)/qdrant_storage:/qdrant/storage:z -d qdrant/qdrant
# !docker run -d -p 9998:9998 apache/tika:latest

skipping.....


In [2]:
from haystack import Pipeline, Document
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.dataclasses import ChatMessage
from haystack.components.writers.document_writer import DuplicatePolicy
from haystack.components.builders import DynamicChatPromptBuilder
from qdrant_haystack import QdrantDocumentStore
from qdrant_haystack.retriever import QdrantRetriever
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import DocumentSplitter
from txtai.pipeline import Textractor
from haystack.components.generators.chat import OpenAIChatGenerator

document_store = QdrantDocumentStore(
    ":memory:",#"http://127.0.0.1",
    recreate_index=True,
    return_embedding=True,
    wait_result_from_api=True,
    index='zarathustra-rag2a'
)

document_embedder = SentenceTransformersDocumentEmbedder(
    model_name_or_path="BAAI/llm-embedder",
    prefix="Represent this document for retrieval: "
)
document_embedder.warm_up()

In [3]:
# %%script echo skipping....
documents = []
textract = Textractor(paragraphs=True)
for paragraph in textract("zarathustra-critical-guide.html"):
    if len(paragraph) > 32:
        documents.append(
            Document(
                meta={'name': "Nietzsche's 'Thus Spoke Zarathustra': A Critical Guide"},
                content=paragraph
            )
        )

for paragraph in textract("zarathustra.md"):
    if len(paragraph) > 32:
        documents.append(
            Document(
                meta={'name': "Thus Spoke Zarathustra"},
                content=paragraph
            )
        )

In [4]:
document_writer = DocumentWriter(document_store = document_store)
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=DocumentCleaner(), name="cleaner")
indexing_pipeline.add_component(instance=document_embedder, name="embedder")
indexing_pipeline.add_component(instance=document_writer, name="writer")
indexing_pipeline.connect("cleaner", "embedder")
indexing_pipeline.connect("embedder", "writer")
indexing_pipeline.draw("indexing_pipeline.png")


In [6]:
# %%script echo skipping.......
indexing_pipeline.run(
    {
        "cleaner": {
            "documents": documents
        },
        "writer": {
            "policy": DuplicatePolicy.OVERWRITE
        }
    }
)

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

3700it [00:01, 2005.75it/s]                          


{'writer': {'documents_written': 3645}}

In [7]:
retriever = QdrantRetriever(
    document_store=document_store,
    top_k=10
)

text_embedder = SentenceTransformersTextEmbedder(
        model_name_or_path="BAAI/llm-embedder",
        prefix="Represent this query for retrieving relevant documents: "
    )

template = """
Given the following information, follow my instruction.

Context: 
{% for document in documents %}
    {{ document.content }}
{% endfor %}

My Instruction: {{ question }}
"""

prompt_builder = DynamicChatPromptBuilder(runtime_variables=["documents"])


In [ ]:
llm = OpenAIChatGenerator(api_base_url="REDACTED", api_key="REDACTED")

In [8]:
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", text_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")
rag_pipeline.draw("rag_pipeline.png")

In [9]:
def send(message, history):
    messages = [ChatMessage.from_user(template)]
    response = rag_pipeline.run(
        {
            "text_embedder": {"text": message},
            "prompt_builder": {
                "template_variables": {"question": message},
                "prompt_source": messages
            }
        }
    )
    return response.get('llm').get('replies')[-1].content

print(send("Who is zarathustra?", None))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



Zarathustra, also known as Zoroaster, is the founder of the ancient Persian religion and the author of its sacred text, the Zend-Avesta. He is believed to have lived in the seventh century BCE and is renowned for his doctrine of the conflict between Ahura Mazda (Ormuzd), the god of light and good, and Angra Mainyu (Ahriman), the god of darkness and evil.

Nietzsche adopted the name Zarathustra for his own hero in his book "Thus Spoke Zarathustra." In Nietzsche's interpretation, Zarathustra is the first to translate morality into the realm of metaphysics and the first to recognize the error of morality. Nietzsche saw Zarathustra as the embodiment of truthfulness and the self-overcoming of morality.

Zarathustra is described as a dancer, a light, a prophet, and a laughing prophet. He is a seer, a wilier, a creator, a future itself, and a bridge to the future. Nietzsche portrays Zarathustra as a genuine, simple man of one meaning and all honesty, a repository of wisdom, a saint of knowle

In [9]:
%%script echo skipping....
import gradio as gr

demo = gr.ChatInterface(fn=send, title="RAG2A")
demo.launch(inline=False)

skipping....


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
